# 4.1张量的结构操作

张量的结构操作主要有：张量的创建，索引切片，维度变换，合并分割

张量的数学运算主要有：标量运算，向量运算，矩阵运算，以及广播机制等．

### 一．张量创建

与numpy创建array的方法很像

In [1]:
import tensorflow as tf
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [2]:
a = tf.constant([1,2,2],dtype=tf.float32)
tf.print(a)

[1 2 2]


In [3]:
b = tf.range(1,10,delta=2)
tf.print(b)

[1 3 5 7 9]


In [4]:
c = tf.linspace(0.,2*3.14,10)
tf.print(c)

[0 0.697777808 1.39555562 ... 4.88444471 5.58222246 6.28]


In [5]:
d = tf.zeros([3,3])
tf.print(d)

[[0 0 0]
 [0 0 0]
 [0 0 0]]


In [6]:
a = tf.ones([3,3])
b = tf.ones_like(a,dtype=tf.float32)
c = tf.zeros_like(b,dtype=tf.float16)
tf.print(a),tf.print(b,c)

[[1 1 1]
 [1 1 1]
 [1 1 1]]
[[1 1 1]
 [1 1 1]
 [1 1 1]] [[0 0 0]
 [0 0 0]
 [0 0 0]]


(None, None)

In [7]:
b = tf.fill([3,2],5)
tf.print(b)

[[5 5]
 [5 5]
 [5 5]]


In [8]:
## 均匀随机分布
tf.random.set_seed(2020)
a = tf.random.uniform([5],minval=0,maxval=10)
tf.print(a)

[7.86967039 9.76694584 1.80886149 1.70957327 8.08133507]


In [9]:
## 正态随机分布
b = tf.random.normal([3,3],mean=0.,stddev=1.)
tf.print(b)

[[0.384146124 -0.61593008 -0.545321405]
 [-0.817522228 1.4578414 -1.65560007]
 [0.245929077 0.279045701 -1.74127507]]


In [10]:
## 正态随机分布，剔除２倍方差以外数据重新生成
ｃ = tf.random.truncated_normal([3,3],mean=0.,stddev=1.,dtype=tf.float32)
tf.print(c)

[[1.26513696 -0.0683450252 -0.182903275]
 [0.362691194 -0.534415722 0.72400105]
 [-0.859625757 -0.0317403711 -0.125741765]]


In [11]:
I = tf.eye(3,3)
tf.print(I)

t = tf.linalg.diag([1,2,3])
tf.print(t)

[[1 0 0]
 [0 1 0]
 [0 0 1]]
[[1 0 0]
 [0 2 0]
 [0 0 3]]


## 二．索引切片

张量的索引方式与numpy几乎是一样的，切片时支持缺省的参数和省略号，对于tf.Variable，可以通过索引和切片对部分元素进行修改．

对于提取张量的连续子区域，可以使用tf.slice.

对于不规则的切片提取，可以使用tf.gather,th.gather_nd,tf.boolean_mask，tf.boolean_mask最为强大，其可以实现tf.gather,th.gather_nd的功能，并且tf.boolean_mask还可以实现布尔索引．

如果要通过修改张量的某些元素得到一个新的张量，可以使用tf.where,tf.scatter_nd

In [12]:
t = tf.random.uniform([5,5],minval=0,maxval=10,dtype=tf.int32)
tf.print(t)

[[6 9 0 3 6]
 [5 0 3 7 3]
 [8 1 8 6 6]
 [2 6 9 2 1]
 [0 0 3 3 5]]


In [13]:
# 首末行
tf.print(t[0],t[-1])
#　第一行第三列
tf.print(t[1,3])
#　第一行至第三行
tf.print(t[1:4,:])
tf.print(tf.slice(t,[1,0],[3,5])) ## tf.slice(input,input_vector,size_vector)
# 第一行至最后一行，第０列至最后一列每隔两列取一列
tf.print(t[1:,:-1:2])

[6 9 0 3 6] [0 0 3 3 5]
7
[[5 0 3 7 3]
 [8 1 8 6 6]
 [2 6 9 2 1]]
[[5 0 3 7 3]
 [8 1 8 6 6]
 [2 6 9 2 1]]
[[5 3]
 [8 8]
 [2 9]
 [0 3]]


In [14]:
## 对于变量来说，还可以使用索引和切片修改部分元素
x = tf.Variable([[1,2],[3,4]],dtype=tf.float32)
tf.print(x)
x[1,:].assign(tf.constant([0.,0.]))
tf.print(x)

[[1 2]
 [3 4]]


<tf.Variable 'UnreadVariable' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [0., 0.]], dtype=float32)>

[[1 2]
 [0 0]]


In [15]:
a = tf.random.uniform([3,3,3],minval=0,maxval=10,dtype=tf.int32)
tf.print(a)

[[[2 7 0]
  [3 2 6]
  [0 8 4]]

 [[8 6 3]
  [6 6 9]
  [0 5 2]]

 [[1 2 3]
  [1 2 9]
  [9 5 7]]]


In [16]:
## 省略号可以表示多个冒号
tf.print(a[...,1])
tf.print(a[:,:,1])

[[7 2 8]
 [6 6 5]
 [2 2 5]]
[[7 2 8]
 [6 6 5]
 [2 2 5]]


In [17]:
## 对于不规则切片
scores = tf.random.uniform([4,10,7],minval=0,maxval=100,dtype=tf.int32)
tf.print(scores)

[[[24 36 9 ... 82 42 13]
  [28 28 76 ... 71 83 29]
  [96 7 78 ... 73 53 8]
  ...
  [15 82 84 ... 5 27 49]
  [13 5 7 ... 15 39 86]
  [45 60 10 ... 3 42 10]]

 [[19 84 80 ... 80 92 77]
  [52 83 40 ... 92 83 45]
  [65 40 67 ... 70 84 29]
  ...
  [63 9 65 ... 3 50 27]
  [47 7 59 ... 56 94 17]
  [71 57 8 ... 48 17 70]]

 [[35 89 12 ... 52 13 88]
  [44 66 97 ... 32 33 29]
  [2 78 93 ... 84 52 24]
  ...
  [40 67 28 ... 35 39 58]
  [5 56 67 ... 63 86 53]
  [83 47 15 ... 79 70 36]]

 [[20 29 23 ... 95 45 59]
  [38 86 19 ... 95 86 95]
  [8 81 32 ... 4 89 45]
  ...
  [52 37 59 ... 64 98 79]
  [2 47 96 ... 60 6 11]
  [41 27 30 ... 83 10 55]]]


In [18]:
## 抽取每个班级第０，５，９个学生的全部成绩
tf.print(tf.gather(scores,[0,5,9],axis=1))

[[[24 36 9 ... 82 42 13]
  [14 94 2 ... 19 96 63]
  [45 60 10 ... 3 42 10]]

 [[19 84 80 ... 80 92 77]
  [68 75 65 ... 72 38 87]
  [71 57 8 ... 48 17 70]]

 [[35 89 12 ... 52 13 88]
  [90 67 2 ... 87 55 78]
  [83 47 15 ... 79 70 36]]

 [[20 29 23 ... 95 45 59]
  [86 35 5 ... 2 42 85]
  [41 27 30 ... 83 10 55]]]


In [20]:
## 抽取每个班级第０个，５个，９个学生的第１，３，６门课程成绩
tf.print(tf.gather(tf.gather(scores,[0,5,9],axis=1),[1,3,6],axis=2))

[[[36 16 13]
  [94 74 63]
  [60 63 10]]

 [[84 98 77]
  [75 94 87]
  [57 25 70]]

 [[89 75 88]
  [67 23 78]
  [47 71 36]]

 [[29 11 59]
  [35 90 85]
  [27 71 55]]]


In [22]:
## 抽取第０个班级第０个学生，第二个班级第４个学生，第３个班级第６个学生的全部成绩
## indices的长度为采样样本的个数，每个元素为采样位置的坐标
tf.print(tf.gather_nd(scores,indices=[(0,0),(2,4),(3,6)]))
tf.print(tf.gather_nd(scores,indices=[(0,0,0),(2,4,0)]))

[[24 36 9 ... 82 42 13]
 [90 1 14 ... 46 28 26]
 [45 76 39 ... 10 83 35]]
[24 90]


以上gather和gather_nd也可以使用boolean_mask来实现

In [24]:
## 抽取每个班级第０，５，９个学生的全部成绩
tf.print(tf.boolean_mask(scores,[True,False,False,False,False,True,False,False,False,True],axis=1))

[[[24 36 9 ... 82 42 13]
  [14 94 2 ... 19 96 63]
  [45 60 10 ... 3 42 10]]

 [[19 84 80 ... 80 92 77]
  [68 75 65 ... 72 38 87]
  [71 57 8 ... 48 17 70]]

 [[35 89 12 ... 52 13 88]
  [90 67 2 ... 87 55 78]
  [83 47 15 ... 79 70 36]]

 [[20 29 23 ... 95 45 59]
  [86 35 5 ... 2 42 85]
  [41 27 30 ... 83 10 55]]]


In [27]:
#抽取第0个班级第0个学生，第2个班级的第4个学生，第3个班级的第6个学生的全部成绩
s = tf.boolean_mask(scores,
    [[True,False,False,False,False,False,False,False,False,False],　##axis=1 第０个班级
     [False,False,False,False,False,False,False,False,False,False],
     [False,False,False,False,True,False,False,False,False,False],
     [False,False,False,False,False,False,True,False,False,False]])
tf.print(s)

[[24 36 9 ... 82 42 13]
 [90 1 14 ... 46 28 26]
 [45 76 39 ... 10 83 35]]


axis给定的前提下，在指定轴上操作，剩余维度取默认；也就是说，给出轴，相当于简略写法，每一个axis的操作是一致的，True代表选择上该行．

In [28]:
## 利用tf.boolean_mask可以实现布尔索引
##　找出矩阵中小于０的元素
c = tf.constant([[-1,1,-1],[2,2,-2],[3,-3,3]],dtype=tf.float32)
tf.print(c,'\n')

tf.print(tf.boolean_mask(c,c<0),'\n')
tf.print(c[c<0])

[[-1 1 -1]
 [2 2 -2]
 [3 -3 3]] 

[-1 -1 -2 -3] 

[-1 -1 -2 -3]


以上方法可以提取张量的部分元素，但是不可以更改张量获取到新的张量，如果要获取到新的张量，可以使用tf.where和tf.scatter_nd.

tf.where可以理解为张量的if版本，此外它还可以用于找到满足条件的所有元素的位置坐标．

tf.scatter_nd的作用和tf.gather_nd相反，tf.scatter_nd可以将某些值插入到一个给定shape的全０张量的指定位置处．

In [29]:
## 找到张量小于０的元素，将其转换为np.nan得到新的张量，与np.where作用类似
d = tf.where(c <0,tf.fill(c.shape,np.nan),c)
tf.print(d)

[[nan 1 nan]
 [2 2 nan]
 [3 nan 3]]


In [30]:
## 如果where只有一个参数，将返回所有满足条件的位置坐标
tf.print(tf.where(c<0))

[[0 0]
 [0 2]
 [1 2]
 [2 1]]


In [34]:
nd = tf.scatter_nd([[0,0],[2,1]],[c[0,0],c[2,1]],c.shape)##将某些位置的元素替换为输入的元素(参数２)，其余元素均为0
tf.print(nd)

[[-1 0 0]
 [0 0 0]
 [0 -3 0]]


In [35]:
## 将张量的某些位置替换为０
tf.print(c - nd)

[[0 1 -1]
 [2 2 -2]
 [3 0 3]]


In [36]:
## 将满足某条件的位置保留，其余设置为０
indices = tf.where(c<0)
tf.scatter_nd(indices,tf.gather_nd(c,indices),c.shape)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-1.,  0., -1.],
       [ 0.,  0., -2.],
       [ 0., -3.,  0.]], dtype=float32)>

## 三．维度变换

tf.reshape 改变张量形状，但是本质上不会改变张量元素的存储顺序，所以操作上会非常迅速，并且是可逆的．张量在内存是线性存储的，同一个层级中的相邻元素的物理位置也相邻．

tf.squeeze　减少维度

tf.expand_dims　增加维度

tf.transpose 交换维度

In [37]:
a = tf.random.uniform(shape=[1,3,3,2],minval=0,maxval=255,dtype=tf.int32)
tf.print(a.shape)
tf.print(a)

TensorShape([1, 3, 3, 2])
[[[[1 191]
   [13 124]
   [45 10]]

  [[127 132]
   [208 233]
   [123 120]]

  [[165 216]
   [170 178]
   [161 118]]]]


In [39]:
b = tf.reshape(a,[3,6])
tf.print(b.shape)
tf.print(b)

TensorShape([3, 6])
[[1 191 13 124 45 10]
 [127 132 208 233 123 120]
 [165 216 170 178 161 118]]


In [40]:
s = tf.squeeze(a)
tf.print(s.shape)
tf.print(s)

TensorShape([3, 3, 2])
[[[1 191]
  [13 124]
  [45 10]]

 [[127 132]
  [208 233]
  [123 120]]

 [[165 216]
  [170 178]
  [161 118]]]


In [41]:
## 在第０维插入长度为１的一个维度
d = tf.expand_dims(s,axis=0)
tf.print(d)

[[[[1 191]
   [13 124]
   [45 10]]

  [[127 132]
   [208 233]
   [123 120]]

  [[165 216]
   [170 178]
   [161 118]]]]


tf.transpose 可以交换张量的维度，与tf.reshpe不同，它会改变张量元素的存储顺序．常用于图片存储格式的交换上．

In [42]:
a = tf.random.uniform(shape=[100,600,600,4],minval=0,maxval=255,dtype=tf.int32)
tf.print(a.shape)

s = tf.transpose(a,perm=[3,1,2,0])
tf.print(s.shape)

TensorShape([100, 600, 600, 4])
TensorShape([4, 600, 600, 100])


## 四.合并分割

和numpy类似，可以使用tf.stack和tf.concat对多个张量进行合并，可以使用tf.split把一个张量分割长多个张量

In [44]:
a = tf.constant([[1.,2.]])
b = tf.constant([[3.,4.]])
c = tf.constant([[5.,6.]])
tf.concat([a,b,c],axis=0)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[1., 2.],
       [3., 4.],
       [5., 6.]], dtype=float32)>

In [45]:
tf.concat([a,b,c],axis=1)

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=array([[1., 2., 3., 4., 5., 6.]], dtype=float32)>

tf.stack会增加一个维度

In [46]:
tf.stack([a,b,c],axis=0)

<tf.Tensor: shape=(3, 1, 2), dtype=float32, numpy=
array([[[1., 2.]],

       [[3., 4.]],

       [[5., 6.]]], dtype=float32)>

In [47]:
tf.stack([a,b,c],axis=1)

<tf.Tensor: shape=(1, 3, 2), dtype=float32, numpy=
array([[[1., 2.],
        [3., 4.],
        [5., 6.]]], dtype=float32)>

In [48]:
a = tf.constant([[1.0,2.0],[3.0,4.0]])
b = tf.constant([[5.0,6.0],[7.0,8.0]])
c = tf.constant([[9.0,10.0],[11.0,12.0]])

c = tf.concat([a,b,c],axis = 0)

tf.split是tf.concat的逆运算，可以指定分割分数平均分割，也可以通过指定每份的记录数量进行分割

In [49]:
tf.split(c,3,axis=0)

[<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[1., 2.],
        [3., 4.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[5., 6.],
        [7., 8.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[ 9., 10.],
        [11., 12.]], dtype=float32)>]

In [50]:
c

<tf.Tensor: shape=(6, 2), dtype=float32, numpy=
array([[ 1.,  2.],
       [ 3.,  4.],
       [ 5.,  6.],
       [ 7.,  8.],
       [ 9., 10.],
       [11., 12.]], dtype=float32)>

In [52]:
tf.split(c,2,axis=1)

[<tf.Tensor: shape=(6, 1), dtype=float32, numpy=
 array([[ 1.],
        [ 3.],
        [ 5.],
        [ 7.],
        [ 9.],
        [11.]], dtype=float32)>,
 <tf.Tensor: shape=(6, 1), dtype=float32, numpy=
 array([[ 2.],
        [ 4.],
        [ 6.],
        [ 8.],
        [10.],
        [12.]], dtype=float32)>]

In [53]:
## 每份的记录数量
tf.split(c,[2,2,2],axis=0)

[<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[1., 2.],
        [3., 4.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[5., 6.],
        [7., 8.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[ 9., 10.],
        [11., 12.]], dtype=float32)>]

# 4.2张量的运算

## 一．标量运算

加减乘除，三角函数，指数，对数，逻辑比较等对张量逐一实施元素运算，有些标量还对常用的数学运算符进行重载，并且支持了numpy的广播机制.

多数都在tf.math模块下

In [58]:
a = tf.constant([[1.,2],[-3,4.]])
b = tf.constant([[5.,6],[7.,8.]])
##　运算符重载
tf.print(a + b)
tf.print(a - b)
tf.print(a * b)
tf.print(a / b)
tf.print(a **2)
tf.print(a **(0.5))
tf.print(tf.sqrt(a))

[[6 8]
 [4 12]]
[[-4 -4]
 [-10 -4]]
[[5 12]
 [-21 32]]
[[0.2 0.333333343]
 [-0.428571433 0.5]]
[[1 4]
 [9 16]]
[[1 1.41421354]
 [-nan 2]]
[[1 1.41421354]
 [-nan 2]]


In [55]:
a // 3
a % 3

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 0.,  0.],
       [-1.,  1.]], dtype=float32)>

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 1.,  2.],
       [-0.,  1.]], dtype=float32)>

In [57]:
a >= 2
(a >= 2) & (a <= 3)
(a >= 2) | (a <= 3)
a == 5

<tf.Tensor: shape=(2, 2), dtype=bool, numpy=
array([[False,  True],
       [False,  True]])>

<tf.Tensor: shape=(2, 2), dtype=bool, numpy=
array([[False,  True],
       [False, False]])>

<tf.Tensor: shape=(2, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True,  True]])>

<tf.Tensor: shape=(2, 2), dtype=bool, numpy=
array([[False, False],
       [False, False]])>

In [59]:
a = tf.constant([1.0,8.0])
b = tf.constant([5.0,6.0])
c = tf.constant([6.0,7.0])
tf.add_n([a,b,c])

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([12., 21.], dtype=float32)>

In [61]:
tf.maximum(a,b)
tf.minimum(a,b)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([5., 8.], dtype=float32)>

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 6.], dtype=float32)>

In [62]:
a = tf.constant([[1.0,8.0],[-1.0,18.0]])
b = tf.constant([[5.0,6.0],[1.0,8.0]])
tf.maximum(a,b) ## 记住是标量运算
tf.minimum(a,b)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 5.,  8.],
       [ 1., 18.]], dtype=float32)>

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 1.,  6.],
       [-1.,  8.]], dtype=float32)>

## 二．向量运算

向量运算只在一个特定轴上运算，将一个向量映射到一个标量或者另一个向量，许多向量运算符都以reduce开头

In [65]:
a = tf.range(1,10)
tf.reduce_sum(a)
tf.reduce_min(a)
tf.reduce_max(a)
tf.reduce_mean(a)
tf.reduce_prod(a)

<tf.Tensor: shape=(), dtype=int32, numpy=45>

<tf.Tensor: shape=(), dtype=int32, numpy=1>

<tf.Tensor: shape=(), dtype=int32, numpy=9>

<tf.Tensor: shape=(), dtype=int32, numpy=5>

<tf.Tensor: shape=(), dtype=int32, numpy=362880>

In [67]:
## 在指定维度上进行
b = tf.reshape(a,(3,3))
b
tf.reduce_sum(b,axis=1,keepdims=True)
tf.reduce_sum(b,axis=0,keepdims=True)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]], dtype=int32)>

<tf.Tensor: shape=(3, 1), dtype=int32, numpy=
array([[ 6],
       [15],
       [24]], dtype=int32)>

<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[12, 15, 18]], dtype=int32)>

In [68]:
## bool类型的reduce
p = tf.constant([True,False,True])
q = tf.constant([False,True])
tf.reduce_all(p)
tf.reduce_any(q)

<tf.Tensor: shape=(), dtype=bool, numpy=False>

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [69]:
## tf.foldr实现tf.reduce_sum
tf.foldr(lambda x,y:x+y,tf.range(10))

<tf.Tensor: shape=(), dtype=int32, numpy=45>

In [72]:
## cum扫描累积
a = tf.range(1,11)
tf.math.cumsum(a)
tf.math.cumprod(a)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 1,  3,  6, 10, 15, 21, 28, 36, 45, 55], dtype=int32)>

<tf.Tensor: shape=(10,), dtype=int32, numpy=
array([      1,       2,       6,      24,     120,     720,    5040,
         40320,  362880, 3628800], dtype=int32)>

In [76]:
## 最大最小索引
tf.argmax(a)
tf.argmin(a)

<tf.Tensor: shape=(), dtype=int64, numpy=9>

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [77]:
## 对张量排序
a = tf.constant([1,3,7,5,4,8])
values,indices = tf.math.top_k(a,3,sorted=True)
values
indices

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([8, 7, 5], dtype=int32)>

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([5, 2, 3], dtype=int32)>